## Environment: 

Python 3.9

## Requirements: 

pip install _gradio opencv-python pillow numpy pillow-lut_

In [1]:
import gradio as gr
from PIL import Image
from pillow_lut import load_cube_file
import os

def apply_lut(image, lut_path):
    lut = load_cube_file(lut_path)
    return image.filter(lut)

def process_image(input_image, lut_option):
    lut_path = f"luts/{lut_option}.cube"
    result = apply_lut(input_image, lut_path)
    return result

# LUT directory
lut_files = [f.split('.')[0] for f in os.listdir('luts') if f.endswith('.cube')]

# Gradio UI
iface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil", label="Upload"),
        gr.Dropdown(choices=lut_files, label="Filters")
    ],
    outputs=gr.Image(type="pil", label="Result"),
    title="Easy Film Sim",
    description="Upload a photo"
)

# Launch UI
iface.launch(share=True)

/Volumes/990Pro/PythonProjects/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: /Volumes/990Pro/PythonProjects/.conda/lib/python3.9/site-packages/gradio/frpc_darwin_arm64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.2
3. Move the file to this location: /Volumes/990Pro/PythonProjects/.conda/lib/python3.9/site-packages/gradio


: 